In [1]:
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np

In [3]:
from importlib import reload

In [4]:
import json

In [5]:
from thesislib.utils import pathutils
from thesislib.utils.ml import report

In [6]:
train_dump = pathutils.get_data_file("prob-synthea-1/output/train.json")
test_dump = pathutils.get_data_file("prob-synthea-1/output/test.json")

train_df = pd.read_json(train_dump)
test_df = pd.read_json(test_dump)

# Idea
- Assume naive bayes
- so :

    $
    Pr(condition|age, gender, race, {symptoms}) = Pr(condition|age) * Pr(condition|gender) * Pr(condition|race) * Pr(condition|{symptoms})
    $
- For the symptoms, we can assume the beta distribution
- Now, because we have quite a bit of data, the simplest solution would be to assume that the beta distribution is in fact your regular bernoulli
- If we stick strictly to the Beta distribution, we would need to evaluate the hyper parameters a and b, a grid search would be good for this
- An experiment should be carried out to estimate values of a and b for different sizes of training data
- But we know that when the dataset becomes large enough, then the effects of a and be become much reduced.
- Underlying assumption here is that we have a "large enough" dataset

For each available condition, we need to evaluate the $\mu$ values that fits the probability distribution for each symptom

In [7]:
symptoms = [itm for itm in train_df.columns if len(itm) == 56]

In [8]:
conditions = train_df['labels'].unique().tolist()

In [9]:
condition_symptom_pr_map = {cnd: {symp: 0 for symp in symptoms} for cnd in conditions}

In [30]:
# fill up the pr map:
grp = train_df.groupby(['labels'])
for itm, _df in grp.__iter__():
    _probs = _df[symptoms].sum()/_df.shape[0]
    condition_symptom_pr_map[itm].update(_probs.to_dict())

In [31]:
# update, added to handle the case of 0 probabilities
eps = 0.001
for cnd in condition_symptom_pr_map:
    for symp in condition_symptom_pr_map[cnd]:
        val = condition_symptom_pr_map[cnd][symp]
        if val < eps:
            val = eps
        elif val > (1 - eps):
            print("hehere", val)
            val = 1 - eps
        condition_symptom_pr_map[cnd][symp] = val

In [32]:
pr_map = condition_symptom_pr_map

In [13]:
# let's do a naive bayes implementation
# note that so far we have ignored the predictive cababilities of the age, race and gender
# ideally we should also attempt to fit some distribution to these two
# a multinomial distribution for race, a gaussian for age and also a binomial for gender
def naive_bayes_evaluation(test_sample, pr_map, symptoms):
    _df = test_sample[symptoms].to_dict()
    cnd_prob = {cnd: 0 for cnd in pr_map}
    predicted = None
    curr_prob = None
    for cnd in pr_map:
        prob = 1.0
        for sym, val in pr_map[cnd].items():
            if _df[sym] == 1:
                prob *= val
            else:
                prob *= (1-val)
        if predicted is None:
            predicted = cnd
            curr_prob = prob
        elif prob > curr_prob:
            curr_prob = prob
            predicted = cnd
        cnd_prob[cnd] = prob
    
    return predicted, cnd_prob

In [14]:
test_predictions = [naive_bayes_evaluation(_itm, condition_symptom_pr_map, symptoms)[0] for idx, _itm in test_df.iterrows()]

In [15]:
test_labels = test_df['labels']
diff = (test_predictions - test_labels) != 0
num_missed = np.sum(diff)
num_labels = len(test_predictions)
accuracy = (num_labels - num_missed) * 1.0/num_labels

In [16]:
print("Test set: Missed %d predictions out of %d samples for an accuracy of %.3f" % (num_missed, num_labels, accuracy))

Test set: Missed 4283 predictions out of 35184 samples for an accuracy of 0.878


This result is better than the results by the RandomForest (86.3%) on the same dataset.

Of course this also fits the data even more perfectly, so it's ability to generalize would be very low. 

One way to attempt to aleviate this potential problem would be to stick with the assumption of the beta distribution for the symptoms. But again, there is still the likelihood of overfitting to the data when we perform a grid search for optimal hyper parameters for the beta distribution.

One other problem with this approach is the fact that $Pr(condition|symptom_i)$ can be 0, a more sensible approach would be to replace all 0's with a very small probability say $0.001$

Also need a much faster implementation for the naive bayes implementation!

Thinking now about the way the data was modelled. The most indicative symptoms for a condition usually occur the least but once they occur it drastically increases the probability that a certain condition is responsible for the other symptoms.
To the best of my knowledge synthea's modelling processes does not capture this thinking, might be good to see/ask how best to make this modelling.

In [17]:
with open(pathutils.get_data_file("prob-synthea-1/output/labels_short_map.json")) as fp:
    label_map = json.load(fp)

In [18]:
# The confusion matrix
_, cnf_mat_str = report.pretty_print_confusion_matrix(test_labels, test_predictions, label_map)

In [19]:
print(cnf_mat_str)

        U    Ast    AS    Ph    Py    AB    ST    CS    Cy
---  ----  -----  ----  ----  ----  ----  ----  ----  ----
U    3557      0     0     0     0     0     0     0   209
Ast     0   3722     0     0     0   216     0    16     0
AS      0     13  2490     1     0    58    89  1295     0
Ph      0     24     6  3856     0    18    60    13     0
Py      1      1     0     0  3742     0     0     0    82
AB      0    474    20    14     0  3312    53    94     0
ST      0      8     0     1     0     0  3903    54     0
CS      0     55  1041     3     0    71   185  2611     0
Cy     86      0     0     0    22     0     0     0  3708


In [ ]:
# can we make a vectorized format of the naive bayes ?
def naive_bayes_evaluation_vec(test_df, pr_map, symptoms):
    _df = test_df[symptoms]
    
    def _xform(item, pr):
        prob = 1.0
        for idx in item.keys():
            val = item.get(idx)
            item.loc[idx] = (pr[idx]*val + (1-pr[idx])*(1-val))
        return item
    
    for cnd in pr_map:
        pr = pr_map[cnd]
        _df_trans = _df.transform(_xform, pr)
    
    cnd_prob = {cnd: 0 for cnd in pr_map}
    predicted = None
    curr_prob = None
    for cnd in pr_map:
        prob = 1.0
        for sym, val in pr_map[cnd].items():
            if _df[sym] == 1:
                prob *= val
            else:
                prob *= (1-val)
        if predicted is None:
            predicted = cnd
            curr_prob = prob
        elif prob > curr_prob:
            curr_prob = prob
            predicted = cnd
        cnd_prob[cnd] = prob
    
    return predicted, cnd_prob

In [ ]:
def _xform(item, pr_map):
    _tf = pd.DataFrame(data={item: 1 for item in pr_map.keys()}, index=[0])
    
    for cnd in pr_map:
        sym_pr = pr_map[cnd]
        prob = 1.0
        for idx in item.keys():
            val = item.get(idx)
            prob *= (sym_pr[idx]*val + (1-sym_pr[idx])*(1-val))
        _tf[cnd] = prob
    return _tf

_ = td.head(1).transform(_xform, axis=1, pr_map=pr_map)

In [135]:
def _xform_1(item):
    _tf = pd.DataFrame(data=[[1 for item in range(len(pr_map))]])
    return _tf

_ = td.head(1).apply(_xform_1, axis=1)

In [142]:
# let's try a different approach ehn
# sklearn has naive bayes (NB) classifiers for Bernoulli like variables, and for gaussian.
# since NB is multiplicative, we should be able to multiply and arrive at the same results!
from sklearn import naive_bayes

In [27]:
train_labels = train_df['labels']
_df = train_df[symptoms]

In [150]:
# the naive bayes classifier
nb_clf = naive_bayes.BernoulliNB(fit_prior=False)
nb_clf.fit(_df, train_labels)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=False)

In [157]:
# predict on the test test
_tf = test_df[symptoms]
test_labels = test_df['labels']

test_predictions_1 = nb_clf.predict(_tf)

In [158]:
diff = (test_predictions_1 - test_labels) != 0
num_missed = np.sum(diff)
num_labels = len(test_predictions_1)
accuracy = (num_labels - num_missed) * 1.0/num_labels
print("Test set: Missed %d predictions out of %d samples for an accuracy of %.3f" % (num_missed, num_labels, accuracy))

Test set: Missed 4283 predictions out of 35184 samples for an accuracy of 0.878


In [159]:
_, cnf_mat_str = report.pretty_print_confusion_matrix(test_labels, test_predictions_1, label_map)
print(cnf_mat_str)

        U    Ast    AS    Ph    Py    AB    ST    CS    Cy
---  ----  -----  ----  ----  ----  ----  ----  ----  ----
U    3557      0     0     0     0     0     0     0   209
Ast     0   3722     0     0     0   216     0    16     0
AS      0     13  2490     1     0    58    89  1295     0
Ph      0     24     6  3856     0    18    60    13     0
Py      1      1     0     0  3742     0     0     0    82
AB      0    474    20    14     0  3312    53    94     0
ST      0      8     0     1     0     0  3903    54     0
CS      0     55  1041     3     0    71   185  2611     0
Cy     86      0     0     0    22     0     0     0  3708


This is the same result as our initial implementation, only faster, way faster!

I believe we can combine different NaiveBayes to make use of all the available data. Would need to study the sklearn implementation of the naive bayes to understand how best to combine them all.

In [205]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils.fixes import logsumexp

class ThesisNaiveBayes:
    def __init__(self, classifier_map):
        self.classifier_map = classifier_map
        self._dfs = [None for idx in range(len(classifier_map))]
        self.fitted = False
        
    def _joint_log_likelihood(self, X):
        if not self.fitted:
            raise ValueError("Model has not been fitted.")
        
        _probs = np.zeros((X.shape[0], self.classes.shape[0]))
        for idx in range(len(self.classifier_map)):
            clf, keys = self.classifier_map[idx]
            _df = X[keys]
            _probs += clf._joint_log_likelihood(_df)
        
        return _probs
    
    def fit(self, X, y):
        labelbin = LabelBinarizer()
        _y = labelbin.fit_transform(y)
        self.classes = labelbin.classes_
        for idx in range(len(self.classifier_map)):
            clf, keys = self.classifier_map[idx]
            _df = X[keys]
            clf.fit(_df, y)
            self.classifier_map[idx][0] =  clf
        
        self.fitted = True
        
    def predict_log_proba(self, X):
        if not self.fitted:
            raise ValueError("Model has not been fitted.")
        
        _probs = self._joint_log_likelihood(X)
        # normalize (see naive_bayes.py in sklearn for explanation!!)
        _log_prob_x = logsumexp(_probs, axis=1)
        return _probs - np.atleast_2d(_log_prob_x).T

    def predict_proba(self, X):
        return np.exp(self.predict_log_proba(X))
    
    def predict(self, X):
        if not self.fitted:
            raise ValueError("Model has not been fitted.")
        jll = self._joint_log_likelihood(X)
        return self.classes[np.argmax(jll, axis=1)]
    

In [53]:
from thesislib.utils.ml import models
from sklearn import naive_bayes

In [36]:
_df = train_df.drop(columns=['labels'])
_tf = test_df.drop(columns=['labels'])

In [54]:
reload(models)

<module 'thesislib.utils.ml.models' from '/Users/teliov/TUD/Thesis/Medvice/Notebooks/thesislib/utils/ml/models.py'>

In [55]:
num_classes = len(conditions)
equal_class_prior = [1.0/num_classes for idx in range(num_classes)]

symptom_gender_clf = naive_bayes.BernoulliNB(fit_prior=False)
race_clf = naive_bayes.MultinomialNB(fit_prior=False)
age_clf = naive_bayes.GaussianNB(priors=equal_class_prior)

symtom_gender = ["gender"] + symptoms

classifier_map = [[symptom_gender_clf, symtom_gender], [race_clf, ["race"]], [age_clf, ["age"]]]
tnb = models.ThesisNaiveBayes(classifier_map)

In [56]:
tnb.fit(_df, train_labels)

In [58]:
diff = (test_predictions_2 - test_labels) != 0
num_missed = np.sum(diff)
num_labels = len(test_predictions_2)
accuracy = (num_labels - num_missed) * 1.0/num_labels
print("Test set: Missed %d predictions out of %d samples for an accuracy of %.3f" % (num_missed, num_labels, accuracy))

Test set: Missed 4233 predictions out of 35184 samples for an accuracy of 0.880


We get a slightly better performance when we include the race, gender and age as part of the predictive properties.

The simplicity of the NaiveBayes allows for a straightforward combination of different individual classifiers.

However, been reading up and while NaiveBayes is a pretty decent classifier especially when you have quite a bit of data, it is not a good estimator. This means that though it would most likely get the correct class more often than not, you can't place much trust in the value of the probabilities that it outputs as a measure of ranking the likelihood of other classes. 

This would not help in our goal of differential diagnosis, we need reliable estimates.

This is something that we would need to verify however!

But in the meantime, we now have an efficient way for vectorizing the computation for Naive Bayes that would definitely scale with the size of the dataset.

Would also be a good idea to study in-depth the implementation of the naive bayes solution in sklearn to pickup some good vectorization skills, the speed up compared to the linear approach I had before is astronomical!!!!